In [ ]:
https://databricks.com/session/scalable-and-incremental-data-profiling-with-spark

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")

In [3]:
from optimus import Optimus

In [4]:
# Create optimus
op = Optimus()

In [6]:
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, IntegerType, ArrayType

df = op.create.df(
            [
                ("words", "str", True),
                ("num", "int", True),
                ("animals", "str", True),
                ("thing", StringType(), True),
                ("second", "int", True),
                ("third", "int", True)
            ],
            [
                ("  I like     fish  ", 1, "dog", "housé", 5 , 1),
                ("    zombies", 2, "cat", "tv", 6, 1),
                ("simpsons   cat lady", 2, "frog", "table", 7, 2),
                (None, 3, "eagle", "glass", 8, 2)
            ])

df.show()

+-------------------+---+-------+-----+------+-----+
|              words|num|animals|thing|second|third|
+-------------------+---+-------+-----+------+-----+
|  I like     fish  |  1|    dog|housé|     5|    1|
|            zombies|  2|    cat|   tv|     6|    1|
|simpsons   cat lady|  2|   frog|table|     7|    2|
|               null|  3|  eagle|glass|     8|    2|
+-------------------+---+-------+-----+------+-----+



### Quantile Statistics

In [7]:
from pyspark.sql import functions as F
#df = df\
#.rows().drop(F.col("billingId")<0)\
#.rows().drop(F.col("price")<0)


In [8]:
print(df.cols().min("billingId"))
print(df.cols().percentile(['billingId', 'price'], [0.05, 0.25, 0.5, 0.75, 0.95]))
print(df.cols().max("billingId"))
print(df.cols().median(["billingId","price"]))
print(df.cols().range(["billingId","price"]))
print(df.cols().std(["billingId","price"]))

ValueError: 'missing_columns' must be 'words', 'num', 'animals', 'thing', 'second', 'third', received '['billingId']'

### Descriptive Statistics

In [15]:
print(df.cols().kurt("billingId"))
print(df.cols().mean("billingId"))
print(df.cols().skewness("billingId"))
print(df.cols().sum("billingId"))
print(df.cols().variance("billingId"))
print(df.cols().mad("billingId"))

-1.0411548120618528
556.0
-0.2137018086949909
10564
78510.55555555555
230.0


In [21]:
df.cols().mad("num")

0.0

In [18]:
df.cols().mad("num", more= True)

{'mad': 0.0, 'median': 2.0}

In [9]:
print(df.cols().percentile(['num'], [0.05, 0.25, 0.5, 0.75, 0.95]))

{0.05: 1.0, 0.25: 1.0, 0.5: 2.0, 0.75: 2.0, 0.95: 3.0}


In [10]:
print(df.cols().mode(["num","second","third"]))

[{'num': 2}, {'second': None}, {'third': [1, 2]}]


### String Operations

In [11]:
df.show() 

+-------------------+---+-------+-----+------+-----+
|              words|num|animals|thing|second|third|
+-------------------+---+-------+-----+------+-----+
|  I like     fish  |  1|    dog|housé|     5|    1|
|            zombies|  2|    cat|   tv|     6|    1|
|simpsons   cat lady|  2|   frog|table|     7|    2|
|               null|  3|  eagle|glass|     8|    2|
+-------------------+---+-------+-----+------+-----+



In [16]:
df\
    .cols().trim("words")\
    .cols().lower("words")\
    .cols().upper(["animals", "thing"])\
    .cols().reverse("thing")\
    .show()

AnalysisException: "cannot resolve '`words`' given input columns: [dummyCol, firstName, product, price, billingId, id, lastName, birth];;\n'Project [id#10, firstName#11, lastName#12, billingId#13, product#14, price#15, birth#16, dummyCol#17, trim('words, None) AS words#551]\n+- AnalysisBarrier\n      +- Filter NOT (price#15 < 0)\n         +- Filter NOT (billingId#13 < 0)\n            +- Relation[id#10,firstName#11,lastName#12,billingId#13,product#14,price#15,birth#16,dummyCol#17] csv\n"

In [11]:
df.cols().iqr("num")

1.0

In [15]:
df.cols().iqr("num", more= True)

{'iqr': 1.0, 'q1': 1.0, 'q3': 2.0}

In [13]:
df.cols().z_score("num").show()

+-------------------+---+-------+-----+------+-----+-----------------+
|              words|num|animals|thing|second|third|        z_col_num|
+-------------------+---+-------+-----+------+-----+-----------------+
|  I like     fish  |  1|    dog|housé|     5|    1|1.224744871391589|
|            zombies|  2|    cat|   tv|     6|    1|              0.0|
|simpsons   cat lady|  2|   frog|table|     7|    2|              0.0|
|               null|  3|  eagle|glass|     8|    2|1.224744871391589|
+-------------------+---+-------+-----+------+-----+-----------------+

